In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/facebook-recruiting-iv-human-or-bot/train.csv.zip')
train
# 정답값 outcome(이상값 탐지대회 --> 신용거래, 보안 분야에 많이 사용)
# 0 : 사람  1: 로봇으로 인식(이상값)

In [ ]:
train['outcome'].value_counts()

In [ ]:
test = pd.read_csv('/kaggle/input/facebook-recruiting-iv-human-or-bot/test.csv.zip')
test

In [ ]:
bids = pd.read_csv('/kaggle/input/facebook-recruiting-iv-human-or-bot/bids.csv.zip')
bids  # 고객 행동 데이터셋

In [ ]:
# 시간처리
# datetime.timestamp()
# from datetime import datetime
# bids['datetime'] = bids['time'].apply(lambda x: datetime.fromtimestamp(x))
# bids

# ValueError: year 309260074 is out of range
# time 컬럼의 단위때문에 범위가 너무 크게 나옴

In [ ]:
#from datetime import datetime
#bids['datetime'] = bids['time'].apply(lambda x: datetime.fromtimestamp(x/1000))
#bids
# 아직 모자름!(범위가 너무 큼)

In [ ]:
# 더 큰수로 나눠줬음
from datetime import datetime
bids['datetime'] = bids['time'].apply(lambda x: datetime.fromtimestamp(x/7000000))
# 날짜는 나중에 실험 ㄱ
bids

In [ ]:
bids['year'] = bids['datetime'].dt.year
bids['month'] = bids['datetime'].dt.month
bids['day'] = bids['datetime'].dt.day
bids['weekday'] = bids['datetime'].dt.weekday

In [ ]:
# train에 묶어줄때 그룹핑해줘서 넣어준다
datetime = bids.groupby('bidder_id')['year','month','day','weekday'].mean()
datetime
# id하나에 여러 정보가 들어있는데 이를 대표값으로 보기위해 평균냈음
# 따로 직접처리를 통해 구체적인 분류 가능!

In [ ]:
# 직접처리
# 원래는 year,month... 등의 날짜정보를 원핫인코딩
# --> 이후 groupby().sum()하면 될 것 같음!
# 하지만 코드가 복잡하고 RAM과부하 위험이 있어 이를 방지하기 위해 다른방법 사요
month = bids.groupby(['bidder_id','month'])['bid_id'].agg(len).unstack().fillna(0)
month
# agg(len)까지의 내용을 train에 추가하기 위해서는 펼쳐야함 --> .unstack()

In [ ]:
bids[bids['bidder_id'] == '8dac2b259fd1c6d1120e519fb1ac14fbqvax8']
# id : 8dac2b259fd1c6d1120e519fb1ac14fbqvax8 에 대해 나온 여러 데이터정보
# 이대로 all_data와 merge하면 데이터가 너무 많아짐
# 각각의 id가 갖는 특성을 하나의 변수로 만들어준다 --> ex) id등장횟수 -> 로봇/사람 구분가능!

In [ ]:
# merchandise는 왜 안쓰는지? -> nunique값이 id에 따라 편차가 커야 하는데
# 전부 1임
# ip는 사람도 여러개일수 있음! + device를 이미 nunique 했음!
bids.groupby('bidder_id')['merchandise'].nunique()

In [ ]:
# id별 구매횟수확인(컬럼 새로추가하는것 아님!)
id_count = bids.groupby('bidder_id')['bid_id'].count().to_frame('id_count')
# to_frame('id_count') : 데이터프레임화한 컬럼이름을 id_count로 설정
id_count
# id_count에는 NULL값이 있음(구매안한 id는 NULL)

In [ ]:
# Auction(쇼핑몰) : 사람은 많은 쇼핑몰에서 구매하지 않지만, 로봇은 많은 쇼핑몰에서 구매한다!
# device(기기) , country 까지 추가
auction_nunique = bids.groupby('bidder_id')['auction','device','country'].nunique()
auction_nunique
# 맨 아래에서 5번째 id --> 637?

In [ ]:
# 이전 구매와 시간 차이가 짧으면 로봇일 것이다!
time_gap = bids.groupby('bidder_id')['time'].apply(lambda x: (x.max()-x.min()) / x.count())

In [ ]:
# url,ip 테스트
# 기계는 url,ip이 많을듯?
url_ip = bids.groupby('bidder_id')['url','ip'].nunique()
url_ip

In [ ]:
all_data = pd.concat([train,test])

In [ ]:
all_data = pd.merge(all_data,datetime,on='bidder_id',how='left')
all_data = pd.merge(all_data,id_count,on='bidder_id',how='left')
all_data = pd.merge(all_data,auction_nunique,on='bidder_id',how='left')
all_data = pd.merge(all_data,time_gap,on='bidder_id',how='left')
all_data = pd.merge(all_data,month,on='bidder_id',how='left')
all_data = pd.merge(all_data,url_ip,on='bidder_id',how='left')
# on = 공통컬럼(기준컬럼)
# how=left -> 왼쪽(all_data)에 id_count(오른쪽)거 merge

In [ ]:
all_data = all_data.fillna(-1)

In [ ]:
# 파생변수
all_data['auction_ratio'] = all_data['id_count'] / all_data['auction']
all_data['device_ratio'] =  all_data['id_count'] / all_data['device']
#all_data['country_ratio'] =  all_data['id_count'] / all_data['country']
# country_ratio는 왜 문제?

In [ ]:
all_data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(16,8))
sns.boxplot(all_data['outcome'],all_data['id_count'],showfliers=False)
# showfliers=False : 이상치를 그림에 보여주지 않겠다!
# id_count높을수록 로봇

In [ ]:
# auction_nunique merge는 왜 유효했나?
plt.figure(figsize=(16,8))
sns.boxplot(all_data['outcome'],all_data['auction'],showfliers=False)

In [ ]:
# time_gap이 의미있었던 이유
plt.figure(figsize=(16,8))
sns.boxplot(all_data['outcome'],all_data['time'],showfliers=False)
# 로봇은 사람보다 확실히 time_gap이 짧다

In [ ]:
# 직접처리가 왜 도움됐는지 확인 --> all_data 컬럼참고해서 y변수 넣어준다
plt.figure(figsize=(16,8))
sns.boxplot(all_data['outcome'],all_data[3],showfliers=False)
# y축 영역 참고하기

In [ ]:
plt.figure(figsize=(16,8))
sns.boxplot(all_data['outcome'],all_data['url'],showfliers=False)
# sns.boxplot(all_data['outcome'],all_data['ip'],showfliers=False)

In [ ]:
all_data.nunique()
# .nunique() : 전체데이터의 종류개수 확인 -> 6713개!
# --> 이상태로 labelencoding 쓰면 점수 낮음

In [ ]:
all_data_2 = all_data.drop(columns= ['bidder_id','outcome'])

In [ ]:
# payment account : 앞부분의 어느정도(5개?)까지는 비슷하다
#                 --> 앞의 문자를 5글자 정도만 가져오자!
all_data_2['payment_account'] = all_data_2['payment_account'].apply(lambda x: x[:5])
all_data_2

In [ ]:
all_data_2['payment_account'].value_counts()
# 은행종류별로 나눠짐을 확인!

In [ ]:
# address도 ㄱㄱ
all_data_2['address'] = all_data_2['address'].apply(lambda x: x[:5])
all_data_2

In [ ]:
# 이제 le 실행가능!
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
all_data_2['payment_account'] = le.fit_transform(list(all_data_2['payment_account']))
all_data_2['address'] = le.fit_transform(list(all_data_2['address']))
all_data_2

In [ ]:
train_2 = all_data_2[:len(train)]
test_2 = all_data_2[len(train):]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=-1,random_state=42,max_depth=7,n_estimators=170)
rfc.fit(train_2,train['outcome'])
result = rfc.predict_proba(test_2)
result

In [ ]:
# 부스팅 모델도 만들어주기
from catboost import CatBoostClassifier
cbc = CatBoostClassifier(verbose=200)
cbc.fit(train_2,train['outcome'])
result_2 = cbc.predict_proba(test_2)

In [ ]:
sub = pd.read_csv('/kaggle/input/facebook-recruiting-iv-human-or-bot/sampleSubmission.csv')
sub

In [ ]:
sub.iloc[:,1] = result_2[:,1] * 0.5 + result[:,1] * 0.5
sub

In [ ]:
sub.to_csv('result.csv',index=False)

In [ ]:
# 점수 향상 아이디어 생각해오기!
# url ip도 nunique 시험해오기!